## This file is dedicated to the analysis of p-pbar production at pp collision, sqrt(s)=5 TeV

In [1]:
import ROOT as r
import numpy as np
import math 
import matplotlib.pyplot as plt

In [2]:
## ROOT table from ALICE data downloaded at this link : https://www.hepdata.net/record/sandbox/1569102768

PbPb_pp_file = r.TFile("ROOTfile/PbPb_pp5tev.root")


In [3]:
Histo = PbPb_pp_file.Get("Table 5")

In [4]:
#Histo.ls() #can be used to visualise the content of the table

In [5]:
P = Histo.Get("Hist1D_y1;1")
TGraphAsymm = Histo.Get("Graph1D_y1;1")

In [6]:
## Now i pay attention to the error that are put into the error bin. In this table, they are 3 different errors

e1 = Histo.Get("Hist1D_y1_e1")  # statistics
e2 = Histo.Get("Hist1D_y1_e2")  # correlated systematics
e3 = Histo.Get("Hist1D_y1_e3")  # uncorrelated systematics

# I define new values for this graph, where i quadratically sum the uncorrelated errors

n = TGraphAsymm.GetN()

for i in range(n):

    err = math.sqrt(e1.GetBinContent(i+1)**2 + e3.GetBinContent(i+1)**2)

    exl = TGraphAsymm.GetErrorXlow(i)
    exh = TGraphAsymm.GetErrorXhigh(i)

    TGraphAsymm.SetPointError(i, exl, exh, err, err)


In [7]:
# Plot of the pp data, the error bar are less important than initially because the TGraph normally have all the errors summed

c = r.TCanvas("c", "test")
c.SetGrid()
c.SetLogy()

TGraphAsymm.SetTitle("p_{T}-distributions of p#bar{p} measured in pp collisions at #sqrt{s} = 5.02 TeV")
TGraphAsymm.GetXaxis().SetTitle("p_{T}[Gev/c]")
TGraphAsymm.GetYaxis().SetTitle("(1/Nev) * d²(N)/dptdy  [(Gev/c)^{⁻1}]")

TGraphAsymm.Draw("AP")

c.Draw()

In [8]:
##The goal here is to FIT every function on the Pb Pb collision data that have a bigger amplitude in terms of value than the pp collision data (more particle created)

In [9]:
## Exponential function, p.18 equation (1) http://arXiv.org/abs/arXiv:1012.3257

fit_min = 0.3
fit_max = 19

m_p = 0.938272


def exp_fct(x, par):
    
    pt = x[0]
    A = par[0]
    T = par[1]
    mp = par[2]

    mt = math.sqrt(pt*pt+mp*mp)

    Factor = A * pt * (1/(T*(mp + T))) 

    Expression = math.exp(-(mt-mp)/T)

    return Factor * Expression



In [10]:
exp = r.TF1("exp1", exp_fct, fit_min, fit_max, 3)

In [11]:
#exp parameters

exp.SetParNames("A", "T", "mp")

exp.SetParameters(10, 0.35, 0.938272)

exp.FixParameter(2, 0.938272)


In [12]:
fit_exp = TGraphAsymm.Fit(exp,"RS0")

****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      827.705
NDf                       =           49
Edm                       =  2.80239e-08
NCalls                    =           94
A                         =      91.0393   +/-   0.911349    
T                         =     0.812703   +/-   0.00700501  
mp                        =     0.938272                      	 (fixed)


In [13]:
## Levy-Tsallis function : p.18 equation (2) http://arXiv.org/abs/arXiv:1012.3257


def levy_fct(x, par):
    
    
    pt = x[0]
    
    dNdy = par[0] 
    T = par[1]
    n = par[2]
    mp = par[3]
    
    
    if (T <= 0) or (n <= 0): #e
        return 0.0
    
    
    mt = math.sqrt( pt*pt + mp*mp ) 
    
    Factor = (n-1)*(n-2)/(n*T*(n*T + mp*(n-2)))

    Arg = 1+((mt-mp)/(n*T))

    Power = Arg**(-n)


    return dNdy * pt * Factor * Power 


In [14]:
# I define a range for the

fit_min = 0.3
fit_max = 20

levy = r.TF1("levy1", levy_fct, fit_min, fit_max, 4)

In [15]:
#Levy parameters

levy.SetParNames("dNdy", "T", "n", "mp")

levy.SetParameters(70, 0.15, 7, 5)  # Initial set of value

levy.SetParLimits(1, 0.08, 0.2)   # T is the Thermal parameter that guide the function at lower pt

#levy.SetParLimits(2, 4, 9)   
#levy.SetParLimits(0, 40, 100)   
#levy.SetParLimits(2, 4, 12)        

## Here for fitting goodnes reason I keep the mass parameter free. The goal was to get the best fit at any cost 
# (for having the best area mesurement for example)

In [16]:
fit_levy = TGraphAsymm.Fit(levy,"RS0")

****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      122.508
NDf                       =           47
Edm                       =  1.57625e-06
NCalls                    =          182
dNdy                      =      75.2721   +/-   0.890725    
T                         =     0.148662   +/-   0.0217758    	 (limited)
n                         =      6.49293   +/-   0.3634      
mp                        =      6.55638   +/-   0.973724    


In [17]:
## Power-law function

def power_law(x, par):

    pt = x[0]

    C = par[0]
    n = par[1]
    p0 = par[2]

    function = 1 + (pt/p0)
    Factor = function**(-n)

    return C * Factor

In [18]:
fit_min = 0.3
fit_max = 20

power_l = r.TF1("power", power_law, fit_min, fit_max, 3)

In [19]:
#power law parameter

power_l.SetParNames("C", "n", "p0")

power_l.SetParameters(10, 6, 2)

power_l.SetParLimits(1, 2.0, 30)

In [20]:
fit_power_l = TGraphAsymm.Fit(power_l,"RS0")

****************************************
         Invalid FitResult  (status = 4 )
****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      461.761
NDf                       =           48
Edm                       =     0.544401
NCalls                    =         1349
C                         =  1.01262e+13   +/-   2.07245e+12 
n                         =      6.60381   +/-   0.0702238    	 (limited)
p0                        =    0.0419452   +/-   0.00259887  


Warning in <Fit>: Abnormal termination of minimization.


In [23]:
# Plot for the exponential fit

canva_exp = r.TCanvas("canvabis", "comparaison")
canva_exp.SetGrid()
canva_exp.SetLogy()

TGraphAsymm.SetTitle("Fit of the Pb-Pb data by exponential function")

TGraphAsymm.GetXaxis().SetTitle("p_{T} [Gev/c]")
TGraphAsymm.GetYaxis().SetTitle("(1/Nev) * d²(N)/dptdy [(Gev/c)^{⁻1}]")

exp.SetLineColor("kRed")
exp.SetLineStyle(1)   

TGraphAsymm.Draw("AP")     
exp.Draw("same")

legend = r.TLegend(0.60,0.68,0.88,0.88)

legend.SetTextFont(42)       
legend.SetTextSize(0.035)    
legend.SetMargin(0.25)       

legend.AddEntry(exp, "Exponential", "l")
legend.AddEntry(TGraphAsymm, "p#bar{p} data", "l")


CHI = str(exp.GetChisquare())[:5]
NDF = str(exp.GetNDF())

Text_exp = r.TPaveText(8,2,13, 20)
Text_exp.AddText(f" #chi^{{2}} = {CHI}")
Text_exp.AddText(f" NdF = {NDF} ")
Text_exp.Draw()

legend.Draw()

canva_exp.Draw()

print(fit_exp)

****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      827.705
NDf                       =           49
Edm                       =  2.80239e-08
NCalls                    =           94
A                         =      91.0393   +/-   0.911349    
T                         =     0.812703   +/-   0.00700501  
mp                        =     0.938272                      	 (fixed)



Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvabis


In [24]:
# Plot for the power-law fit

canva_pl = r.TCanvas("canvabis", "comparaison")
canva_pl.SetGrid()
canva_pl.SetLogy()

TGraphAsymm.SetTitle("Fit of the p#bar{p} data by a power-law function")
TGraphAsymm.GetXaxis().SetTitle("p_{T} [Gev/c]")
TGraphAsymm.GetYaxis().SetTitle("(1/Nev) * d²(N)/dptdy [(Gev/c)^{⁻1}]")

power_l.SetLineColor("kGreen+2")
power_l.SetLineStyle(3) 


TGraphAsymm.Draw("AP")     
power_l.Draw("same")


legend = r.TLegend(0.60,0.68,0.88,0.88)

legend.SetTextFont(42)       
legend.SetTextSize(0.035)    
legend.SetMargin(0.25)       

legend.AddEntry(power_l, "Power-law", "l")
legend.AddEntry(TGraphAsymm, "p#bar{p} data", "l")

CHI_pl = str(power_l.GetChisquare())[:5]
NDF_pl = str(power_l.GetNDF())

Text_power_l = r.TPaveText(8, 2 ,13, 20)
Text_power_l.AddText(f" #chi^{{2}} = {CHI_pl}")
Text_power_l.AddText(f" NdF = {NDF_pl} ")
Text_power_l.Draw()

legend.Draw()

canva_pl.Draw()

print(fit_power_l)

****************************************
         Invalid FitResult  (status = 4 )
****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      461.761
NDf                       =           48
Edm                       =     0.544401
NCalls                    =         1349
C                         =  1.01262e+13   +/-   2.07245e+12 
n                         =      6.60381   +/-   0.0702238    	 (limited)
p0                        =    0.0419452   +/-   0.00259887  



Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvabis


In [26]:
# Plot for the levy-tsallis fit

canva_levy = r.TCanvas("canva_levy", "Levy")
canva_levy.SetGrid()
canva_levy.SetLogy()

TGraphAsymm.SetTitle("Fit of the p#bar{p} data by a Levy-Tsalli function")
TGraphAsymm.GetXaxis().SetTitle("p_{T} [Gev/c]")
TGraphAsymm.GetYaxis().SetTitle("(1/Nev) * d²(N)/dptdy [(Gev/c)^{⁻1}]")

levy.SetLineColor("kBlue")
levy.SetLineStyle(2)      

TGraphAsymm.Draw("AP")     
levy.Draw("same")


legend = r.TLegend(0.60,0.68,0.88,0.88)

legend.SetTextFont(42)       
legend.SetTextSize(0.035)   
legend.SetMargin(0.25)       

legend.AddEntry(levy, "Levy-Tsallis","l")
legend.AddEntry(TGraphAsymm, "p#bar{p} data", "l")


CHI_levy = str(levy.GetChisquare())[:8]
NDF_levy = str(levy.GetNDF())

Text_levy = r.TPaveText(8,2,13, 20)
Text_levy.AddText(f" #chi^{{2}} = {CHI_levy}")
Text_levy.AddText(f" NdF = {NDF_levy} ")
Text_levy.Draw()


legend.Draw()

canva_levy.Draw()

print(fit_levy)

****************************************
Minimizer is Minuit2 / Migrad
Chi2                      =      122.508
NDf                       =           47
Edm                       =  1.57625e-06
NCalls                    =          182
dNdy                      =      75.2721   +/-   0.890725    
T                         =     0.148662   +/-   0.0217758    	 (limited)
n                         =      6.49293   +/-   0.3634      
mp                        =      6.55638   +/-   0.973724    



Warning in <TCanvas::Constructor>: Deleting canvas with same name: canva_levy
